In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.preprocessing import MinMaxScaler
import librosa

In [2]:
df = pd.read_csv(r"C:\Users\user\Desktop\Project\EXPERIMENTS\revised-data\audio_features_cp.csv")

# Separate features and labels
x_cols = ['mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 
           'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13', 'spectral_centroid', 
           'spectral_bandwidth', 'zero_crossing_rate', 'spectrogram_mean' , 'spectrogram_median' , 'spectrogram_variance']
y_cols = ['label']

# Scale only feature columns
SMM = MinMaxScaler(feature_range=(0, 1))
df[x_cols] = SMM.fit_transform(df[x_cols])

X = df[x_cols]
y = df[y_cols].values.ravel()  # Convert to 1D array for the model

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [4]:
model = models.Sequential()
model.add(layers.InputLayer(input_shape=(19,)))

# Reshape input to be suitable for convolution (like an image with 1 channel)
model.add(layers.Reshape((19, 1, 1)))  # Reshape to (19, 1, 1) for CNN processing

# 1st Convolutional Layer
model.add(layers.Conv2D(32, kernel_size=(3, 1), activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 1), padding='same'))

# 2nd Convolutional Layer
model.add(layers.Conv2D(64, kernel_size=(3, 1), activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 1), padding='same'))

# Flattening the output from the convolutional layers to pass into Dense layers
model.add(layers.Flatten())

# Fully Connected Layer
model.add(layers.Dense(128, activation='relu'))

# Adding Dropout to prevent overfitting
model.add(layers.Dropout(0.2))

# Output Layer for binary classification (scream or non-scream)
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [6]:
model.fit(X_train, y_train, epochs=50)

Epoch 1/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8898 - loss: 0.2707
Epoch 2/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8937 - loss: 0.2626
Epoch 3/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9086 - loss: 0.2453
Epoch 4/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9005 - loss: 0.2646
Epoch 5/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9253 - loss: 0.2064
Epoch 6/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9137 - loss: 0.2211
Epoch 7/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9159 - loss: 0.2190
Epoch 8/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9223 - loss: 0.2100
Epoch 9/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9182 - loss: 0.2063
Epoch 10/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9183 - loss: 0.2012
Epoch 11/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9272 - loss: 0.1872
Epoch 12/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9272 - lo

In [7]:
model.evaluate(X_test, y_test)

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9229 - loss: 0.2115


[0.24175170063972473, 0.9136363863945007]

In [12]:
### Prediction Function for New Audio
def extract_features(audio_file):
    # Load the audio file
    y, sr = librosa.load(audio_file, sr=None)
    
    # Extract MFCC (first 13 coefficients)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    
    # Extract Spectral Centroid
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    
    # Extract Spectral Bandwidth
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    
    # Extract Zero-Crossing Rate
    zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=y))
    
    # Extract Spectrogram (STFT)
    spectrogram = np.abs(librosa.stft(y))
    spectrogram_mean = np.mean(spectrogram.T, axis=0)  # Get mean of spectrogram along time axis
    spectrogram_mean2 = np.mean(spectrogram_mean)
    spectrogram_median = np.median(spectrogram_mean)
    spectrogram_variance = np.var(spectrogram_mean)
    
    # Return the features as a list (order must match the training features)
    return [
        mfccs_mean[0], mfccs_mean[1], mfccs_mean[2], mfccs_mean[3], mfccs_mean[4],
        mfccs_mean[5], mfccs_mean[6], mfccs_mean[7], mfccs_mean[8], mfccs_mean[9],
        mfccs_mean[10], mfccs_mean[11], mfccs_mean[12],
        spectral_centroid, spectral_bandwidth, zero_crossing_rate,
        spectrogram_mean2, spectrogram_median, spectrogram_variance
    ]

# Prediction function
def pred(aud):
    audio = aud
    features = extract_features(audio)
    feature = np.array(features).reshape(1, 19)  # Correct shape
    feature = SMM.transform(feature)  # Use transform instead of fit_transform
    predis = model.predict(feature)
    
    if (predis > 0.5):
        print("Screaming")
    else:
        print("Non_screaming")
        
    print(predis)

In [19]:
ipt = r"C:\Users\user\Desktop\Project\EXPERIMENTS\testaudio\test5.wav"
pred(ipt)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Non_screaming
[[0.262057]]


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
